# Predicting continuous disorder

Try dataset from [SETH](https://www.biorxiv.org/content/10.1101/2022.06.23.497276v1.full.pdf) with different architectures.

### Data

We contacted Rostlab, and they released their 1174 entry training dataset. As validation set, we use 15% of that i.e. 176 sequences.
One issue is that they have an ignore index (999) but PyTorch's MSELoss does not support that currently. Therefore, we built this ourselves.